In [11]:
# @title Install Dependancies

# !pip install adlfs > /dev/null 2>&1 && echo $?
# !pip install pyarrow > /dev/null 2>&1 && echo $?
# !pip install praw > /dev/null 2>&1 && echo $?
# !pip install transformers > /dev/null 2>&1 && echo $?
# !pip install retry > /dev/null 2 > & 1 & & echo $?

In [12]:
# @title Import Dependancies


import time
from retry import retry
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random
import pyarrow
import requests
from PIL import Image
import os
import datetime as dt
import os
from datetime import datetime
import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm
from adlfs import AzureBlobFileSystem
import os
import hashlib
import os
from datetime import datetime
from io import BytesIO
import random
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from retry import retry
from retry.api import retry_call
import requests
from PIL import Image
import json

In [13]:
# @title Initialize Secrets

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)',
								  check_for_async=False)

os.environ["AZURE_STORAGE_ACCOUNT_NAME"] = "ajdevreddit"

os.environ[
	"AZURE_STORAGE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

os.environ[
	"AZURE_STORE_ACCOUNT_KEY"] = "+9066TCgdeVignRdy50G4qjmNoUJuibl9ERiTGzdV4fwkvgdV3aSVqgLwldgZxj/UpKLkkfXg+3k+AStjFI33Q=="

In [14]:
# @title Initalize Helpers


class BlipCaption:
	def __init__(self, device_name: str = "cuda"):
		self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
		self.device = torch.device(device_name)
		self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large",
																  torch_dtype=torch.float16).to(self.device)

	def caption_image(self, image_path: str) -> str:
		try:
			raw_image = Image.open(image_path).convert('RGB')
			inputs = self.processor(raw_image, return_tensors="pt").to(self.device, torch.float16)
			out = self.model.generate(**inputs)
			return self.processor.decode(out[0], skip_special_tokens=True, max_new_tokens=50)

		except Exception as e:
			print(e)
			return ""


class Functions(object):
	@retry(Exception, tries=5, delay=1, jitter=1)
	def make_trouble(self, url):
		return requests.get(url)

	def get_hash_from_path(self, in_path: str):
		if os.path.exists(in_path):
			with open(in_path, 'rb') as f_:
				content = f_.read()
				result = hashlib.md5(content).hexdigest()
				return result, content
		else:
			return ""

	def fetch_image(self, x: object, file_list__, file_system__) -> str:
		try:
			url = x['original_url']
			subreddit = x['subreddit']
			image_id = x['id']
			os.makedirs(f"temp/image/{subreddit}", exist_ok=True)
			temp_path = f"temp/image/{subreddit}/{image_id}.jpg"
			out_path = f"data/image/{image_id}.jpg"
			if file_system__.exists(out_path) and file_system__.size(out_path) > 0:
				print(f"exists {image_id}, downloading to {temp_path}")
				file_system__.download(out_path, temp_path)
			else:
				print(f"File is empty for {image_id}")
				file_system__.delete(out_path)
			if os.path.exists(temp_path):
				md5, content = self.get_hash_from_path(temp_path)
				if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
					if out_path in file_list__:
						return out_path
					else:
						file_system__.upload(temp_path, out_path, overwrite=True)
						return out_path
				else:
					return ""
			else:
				response = self.make_trouble(url)
				md5 = hashlib.md5(response.content).hexdigest()
				if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
					try:
						raw_image = Image.open(BytesIO(response.content))
						raw_image.save(temp_path)
						raw_image.close()
						if out_path in file_list__:
							return out_path
						else:
							file_system__.upload(temp_path, out_path)
							return out_path
					except Exception as ex:
						message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
						print(message)
						return ""
				else:
					return ""
		except Exception as ex:
			message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
			print(message)
			return ""

	def get_name_for_image(self, x: object, file_list__) -> str:
		path = x['path']
		if path != "" and path in file_list__:
			return os.path.basename(path)
		else:
			return ""

	def set_exists(self, x: object, file_system__) -> bool:
		try:
			sub_reddit = x['subreddit']
			record_id = x['id']
			remote_path = x['path']
			return file_system__.exists(remote_path)
		except Exception as ex:
			return False

	def set_hash(self, x: object):
		sub_reddit = x['subreddit']
		record_id = x['id']
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"
		if os.path.exists(temp_path):
			return hashlib.md5(open(temp_path, 'rb').read()).hexdigest()
		else:
			return ""

	def add_source(self, x: object, source_list) -> str:
		sub_reddit = x['subreddit']
		for source in source_list:
			data_source = source['data']
			source_name = source['name']
			if sub_reddit in data_source:
				return source_name
		return ""

	def write_log_message(self, submission_id: str, subreddit: str, message: str, exception: Exception) -> str:
		date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
		return f"{date_time}\t{subreddit}\t{submission_id}\t{message}\t{exception}\n"

	def apply_caption(self, x: object, caption_routine: list[BlipCaption], file_system: AzureBlobFileSystem) -> str:
		exists = x['exists']
		if not exists:
			return ""
		sub_reddit = x['subreddit']
		record_id = x['id']
		remote_path = x['path']
		os.makedirs(f"temp/image/{sub_reddit}/", exist_ok=True)
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"

		if os.path.exists(temp_path):
			try:
				result = random.choice(caption_routine).caption_image(temp_path)
				return result
			except Exception as ex:
				message = self.write_log_message(x['id'], x['subreddit'], "Failure in apply_caption", ex)
				print(message)
				return ""
		else:
			try:
				file_system.download(remote_path, temp_path)
				result = random.choice(caption_routine).caption_image(temp_path)
				return result
			except Exception as ex:
				message = self.write_log_message(x['id'], x['subreddit'], "Failure in apply_caption", ex)
				print(message)
				return ""

	def fix_path(self, x: object, fl) -> str:
		current_path = x['path']
		exists = x['exists']
		if current_path in fl:
			return current_path
		else:
			image_id = x['id']
			if exists:
				return f"data/image/{image_id}.jpg"
			else:
				return ""
		current_path = x['path']
		exists = x['exists']
		if current_path in fl:
			return current_path
		else:
			image_id = x['id']
			if exists:
				return f"data/image/{image_id}.jpg"
			else:
				return ""


class AzureFileStorageAdapter(object):
	def __init__(self, container_name: str = "data"):
		self.__account_name: str = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
		self.__account_key: str = os.environ["AZURE_STORAGE_ACCOUNT_KEY"]
		self.container_name: str = container_name

	def get_file_storage(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(
			account_name=self.__account_name,
			account_key=self.__account_key,
			container_name=self.container_name)

	def get_file_storage_root(self) -> AzureBlobFileSystem:
		return AzureBlobFileSystem(account_name=self.__account_name, account_key=self.__account_key,
								   container_name=self.container_name)


schema = pyarrow.schema(
	[
		pyarrow.field("id", pyarrow.string()),
		pyarrow.field("subreddit", pyarrow.string()),
		pyarrow.field("author", pyarrow.string()),
		pyarrow.field("title", pyarrow.string()),
		pyarrow.field("caption", pyarrow.string()),
		pyarrow.field("hash", pyarrow.string()),
		pyarrow.field("permalink", pyarrow.string()),
		pyarrow.field("original_url", pyarrow.string()),
		pyarrow.field("image_name", pyarrow.string()),
		pyarrow.field("path", pyarrow.string()),
		pyarrow.field("model", pyarrow.string()),
		pyarrow.field("exists", pyarrow.bool_()),
		pyarrow.field("curated", pyarrow.bool_()),
		pyarrow.field("accept", pyarrow.bool_()),
		pyarrow.field("tags", pyarrow.list_(pyarrow.string()))
	]
)

In [15]:
# @title Image Acquisition Flow

display("==== Starting Image Acquisition ====")

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

'==== Starting Image Acquisition ===='

In [16]:
# @title Set Targets


# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture", "trippinthroughtime", "bathandbodyworks",	"mildlypenis",	"fatsquirrelhate",
subs = [
	"tightdresses",
	"Dresses",
	"SlitDresses",
	"CollaredDresses",
	"DressesPorn",
	"WomenInLongDresses",
	"TrueFMK",
	"DLAH",
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"AsianInvasion",
	"AesPleasingAsianGirls"
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
]

display("== Current Subs ==")
display(json.dumps(subs))

'== Current Subs =='

'["tightdresses", "Dresses", "SlitDresses", "CollaredDresses", "DressesPorn", "WomenInLongDresses", "TrueFMK", "DLAH", "SFWRedheads", "sfwpetite", "SFWNextDoorGirls", "realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion", "AesPleasingAsianGirlssexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls"]'

In [17]:
# @title Set Sources


sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyDressDiffusion", "data": ["SunDressesGoneWild", "ShinyDresses", "SlitDresses", "CollaredDresses", "DressesPorn","WomenInLongDresses", "Dresses"]},
	{"name": "SexyAsianDiffusion", "data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion", "AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion", "data": ["sexygirls", "PrettyGirls", "gentlemanboners" "hotofficegirls", "tightdresses", "DLAH", "TrueFMK"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

display("== Sources To Apply ==")
display(sources_df.shape)
display(sources_df)

'== Sources To Apply =='

(20, 2)

,name,data
0,CityDiffusion,[CityPorn]
1,NatureDiffusion,[EarthPorn]
2,CosmicDiffusion,[spaceporn]
3,ITAPDiffusion,[itookapicture]
4,MemeDiffusion,[memes]
5,TTTDiffusion,[trippinthroughtime]
6,WallStreetDiffusion,[wallstreetbets]
7,SexyDiffusion,"[selfies, Amicute, amihot, AmIhotAF, HotGirlNe..."
8,FatSquirrelDiffusion,[fatsquirrelhate]
9,CelebrityDiffusion,[celebrities]


In [18]:
# @title Read primary_caption


extant_data = pandas.read_parquet("data/parquet/primary_caption.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True, drop=False)

display("== primary_caption ==")
display(extant_data.shape)
display(extant_data.head())

'== primary_caption =='

(45600, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]


In [19]:
# @title Create Temp Dirs


for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 20/20 [00:00<00:00, 3333.71it/s]


In [20]:
# @title Locate And Cache Reddit Data


data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

try:
	cache_data = pandas.DataFrame(data)
	cache_data.set_index('id', drop=False, inplace=True)
except KeyError:
	cache_data = pandas.DataFrame(data=[{'id': "foo"}])
	cache_data.set_index('id', drop=False, inplace=True)

for sub in subs:
	subreddit_stream_2 = list(reddit.subreddit(display_name=sub).hot(limit=100))
	subreddit_stream_3 = list(reddit.subreddit(display_name=sub).new(limit=100))
	temp_dir_path = f"temp/{sub}"
	remote_temp_data = f"data/temp/{sub}"
	for time_filter in ['day', 'year', 'week', 'all', 'month']:
		try:
			subreddit_stream = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
			subreddit_stream = list(subreddit_stream)

			all_streams = subreddit_stream + subreddit_stream_2 + subreddit_stream_3
			all_streams = list(set(all_streams))
			p = tqdm(all_streams, total=len(all_streams), desc=f"Posts - {sub} - {time_filter}")
			for submission in p:
				if submission is None:
					continue
				else:
					if submission.id in extant_data.index.values or submission.id in cache_data.index.values:
						continue
					else:
						try:
							if file_system.exists(f"{remote_temp_data}/{submission.id}.parquet"):
								file_system.download(f"{remote_temp_data}/{submission.id}.parquet",
													 f"{temp_dir_path}/{submission.id}.parquet")
								continue

							sub_id = submission.id
							author_name = 'Unknown'
							subreddit_name = sub
							sub_title = submission.title
							perma_link = submission.permalink
							sub_url = submission.url
							try:
								author_name = submission.author.name
							except Exception as e:
								author_name = 'Unknown'
								pass
							p = {
								'id': sub_id,
								'subreddit': subreddit_name,
								'author': author_name,
								'title': sub_title,
								'caption': '',
								'hash': '',
								'permalink': perma_link,
								'original_url': sub_url,
								'image_name': '',
								'path': '',
								'thumbnail_path': '',
								'exists': False,
								'curated': False,
								'Tags': ['']
							}
							d = pd.DataFrame([p])
							d.to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
							d.to_parquet(f"{remote_temp_data}/{submission.id}.parquet", filesystem=file_system)
							continue
						except Exception as e:
							log = f"{submission.id}, {sub}, Error Writing Post, {e}"
							print(log)
							continue
		except Exception as e:
			log = f"{sub}, Error Getting Posts For SubReddit, {e}"
			print(log)
			continue

Posts - AsianInvasion - month: 100%|██████████| 101/101 [00:00<00:00, 167.13it/s]


NotFound: received 404 HTTP response

In [ ]:
# @title Read Cache And Stage Images

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data.head())

In [ ]:
# @title Apply Filter For Images and Existance On Data

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & (temp_data['original_url'].str.endswith('.jpg') | temp_data['original_url'].str.endswith('.png'))]

filtered.dropna(axis=1, how='all')

filtered.reset_index(drop=True, inplace=True)

filtered.set_index('id', inplace=True, drop=False)

display("==== FILTERED DATA ====")

display(filtered.shape)

display(filtered.head())

In [ ]:
# @title Add Source To Model

with tqdm(total=len(filtered)) as pbar:
	for i, row in filtered.iterrows():
		d = functions.add_source(row, sources)
		filtered.loc[i, 'model'] = d
		pbar.update()

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered.head())

In [ ]:
@retry(Exception, tries=10, delay=3, jitter=2)
def make_trouble(url):
	return requests.get(url)

def get_hash_from_path(in_path: str):
	if os.path.exists(in_path):
		with open(in_path, 'rb') as f_:
			content = f_.read()
			result = hashlib.md5(content).hexdigest()
			return result, content
	else:
		return ""


def fetch_image(x: object, file_list__, file_system__) -> str:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		image_id = x['id']
		os.makedirs(f"temp/image/{subreddit}", exist_ok=True)
		temp_path = f"temp/image/{subreddit}/{image_id}.jpg"
		out_path = f"data/image/{image_id}.jpg"
		if file_system__.exists(out_path):
			file_system__.download(out_path, temp_path)
		if os.path.exists(temp_path):
			md5, content = get_hash_from_path(temp_path)
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				if out_path in file_list__:
					file_system.download(out_path, temp_path)
					return out_path
				else:
					return out_path
			else:
				print("Invalid")
				return ""
		else:
			response = requests.get(url)
			md5 = hashlib.md5(response.content).hexdigest()
			if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
				try:

					raw_image = Image.open(requests.get(url, stream=True).raw)
					if raw_image.mode in ("RGBA", "P"):
						raw_image = raw_image.convert("RGB")
					raw_image.save(temp_path)
					raw_image.close()
					if out_path in file_list__:
						return out_path
					else:
						file_system__.upload(temp_path, out_path)
						return out_path
				except Exception as ex:
					message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}, {url}"
					print(message)
					return ""
			else:
				return ""
	except Exception as ex:
		message = f"{x['id']}, {x['subreddit']}, Failure in fetch_image, {ex}"
		print(message)
		return ""

In [ ]:
# @title Fetch Images

file_list = file_system.ls("data/image")

for elem in tqdm(file_list, total=len(file_list)):
	if file_system.size(elem) == 0:
		print(f"I am empty {elem}")
		file_system.delete(elem)
		file_list.remove(elem)

In [ ]:
file_list = file_system.ls("data/image")
with tqdm(total=len(filtered)) as pbar:
	for i, row in filtered.iterrows():
		d = fetch_image(row, file_list, file_system)
		filtered.loc[i, 'path'] = d
		pbar.update()

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered.head())

In [ ]:
def set_exists(x: object, file_system__) -> bool:
	try:
		sub_reddit = x['subreddit']
		record_id = x['id']
		remote_path = x['path']
		temp_path = f"temp/image/{sub_reddit}/{record_id}.jpg"
		if os.path.exists(temp_path) and file_system__.exists(remote_path):
			print("found in both local and remote")
			return True
		if os.path.exists(temp_path) and not file_system__.exists(remote_path):
			file_system__.upload(temp_path, remote_path)
			print("Only found in local, downloading to remote")
			return True
		if not os.path.exists(temp_path) and file_system__.exists(remote_path):
			file_system__.download(remote_path, temp_path)
			print("Only found in remote, downloading to local")
			return True
		else:
			print("Not found")
			return False
	except Exception as ex:
		print(ex)
		return False

In [ ]:
# @title Apply Exist

for i, row in filtered.iterrows():
	d = set_exists(row, file_system)
	filtered.loc[i, 'exists'] = d

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered.head())

In [ ]:
# @title Apply Hash

for i, row in filtered.iterrows():
	d = functions.set_hash(row)
	filtered.loc[i, 'hash'] = d
display(filtered.shape)
display(filtered)

In [ ]:
# @title Filter On Exists and path existance

ts = datetime.today().timestamp()

exists = filtered.loc[(filtered['exists'] == True) & (filtered['path'] != "")]
exists.dropna(axis=1, how='all')
exists.reset_index(drop=True, inplace=True)

display(exists.shape)
display(exists.head())

new_records_to_process = exists.copy()

In [ ]:
# @title Store New Records


out_name = f"data/temp/caption/primary/{ts}_new_records_to_process.parquet"
new_records_to_process.to_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)

new_df = pandas.read_parquet(out_name, engine='pyarrow', filesystem=file_system, schema=schema)

display("== New Records To Process ==")
display(new_df.shape)
display(new_df)

In [ ]:
# @title End Cache Collection


display(f"Total Number Of New Images - {new_df.shape[0]}")
display("0-1 Image Acquisition Complete - Writing new output and Shutting Down")

In [ ]:
# @title Import BLIP


import torch

caption_0 = BlipCaption("cuda")
# caption_1 = BlipCaption("cuda")

In [ ]:
# @title Read Temp Captions

ready_to_caption = pandas.read_parquet('data/temp/caption/primary', engine='pyarrow', filesystem=file_system, schema=schema)
ready_to_caption.set_index("id", inplace=True, drop=False)
ready_to_caption.drop_duplicates(subset=['id'], keep='last', inplace=True)

extant_curate = pandas.read_parquet('data/parquet/primary_caption.parquet', engine='pyarrow', filesystem=file_system, schema=schema)
extant_curate.set_index("id", inplace=True, drop=False)

display("=== Extant Data ===")
display(extant_curate.shape)
display(extant_curate.head())

display("=== Read To Caption ===")
display(ready_to_caption.shape)
display(ready_to_caption.head())

In [ ]:
# @title Filter on bad data


check = ready_to_caption.loc[(ready_to_caption['caption'] == "") & (ready_to_caption['caption'].notnull()) & (ready_to_caption['exists'] == True) & (~ready_to_caption['id'].isin(extant_curate.index.values))]

ready_to_caption = check.copy()

display("== Filtered On Existing Caption ==")
display(ready_to_caption.shape)
display(ready_to_caption.head())

In [ ]:
# @title Apply Captions

with tqdm(total=len(ready_to_caption)) as pbar:
	for i, row in ready_to_caption.iterrows():
		ready_to_caption.loc[i, 'caption'] = functions.apply_caption(row, [caption_0, caption_0], file_system)
		pbar.update()

display("== Data With Captions ==")
display(ready_to_caption.shape)
display(ready_to_caption.head())

In [ ]:
# @title Drop Missing Captions

dropped = ready_to_caption.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)
dropped.set_index("id", inplace=True, drop=False)

post_caption_drop = dropped.copy()

display("== Dropped And Reset Data After Captioning ==")
display(post_caption_drop.shape)
display(post_caption_drop.head())

In [ ]:
# @title Ensure Valid

foo = extant_curate.index.values
bar = post_caption_drop.index.values

display(f"Extant Contains New: {foo in bar} - {bar in foo}")

In [ ]:
# @title Combine extant_curate and post_caption_drop

concat = pandas.concat([extant_curate, post_caption_drop])

dropped = concat.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)
dropped.set_index("id", inplace=True, drop=False)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

In [ ]:
# @title Create Backup

back_up_name = f"data/parquet/primary_caption_{dt.datetime.timestamp(dt.datetime.now())}.parquet"

display(f"== Writing Back-Up {back_up_name} ==")
current = pandas.read_parquet('data/parquet/primary_caption.parquet', engine='pyarrow', filesystem=file_system,
							  schema=schema)
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

display(current.shape)
display(current.head())

In [ ]:
#@title Write Output

dropped.reset_index(inplace=True, drop=True)
dropped.to_parquet("data/parquet/primary_caption.parquet", schema=schema, filesystem=file_system)

display("== Output Written ==")
display(dropped.shape)
display(dropped)

In [ ]:
# @title Plot Summary: By Subreddit

primary_caption = pandas.read_parquet("data/parquet/primary_caption.parquet", engine='pyarrow', filesystem=file_system)
accepted_primary = primary_caption.where(primary_caption['accept'] == True)
accepted_primary.dropna(inplace=True)
accepted_primary.reset_index(inplace=True, drop=True)

thumbnail_curation = pandas.read_parquet('data/parquet/thumbnail_curation.parquet', engine='pyarrow',
										 filesystem=file_system)
accepted_thumbnail = thumbnail_curation.where(thumbnail_curation['thumbnail_accept'] == True)
accepted_thumbnail.dropna(inplace=True)
accepted_thumbnail.reset_index(inplace=True, drop=True)

In [ ]:
group = accepted_primary[["id", "subreddit"]].groupby("subreddit").count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="Subreddits Count Of Accepted Images", legend=True)
display(plot)

group = primary_caption[["id", "subreddit"]].groupby("subreddit").count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="Subreddits Count Of ALL Images", legend=True)
display(plot)

group = accepted_thumbnail[["id", "subreddit"]].groupby("subreddit").count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="Curated Count of Accepted Images By Sub", legend=True)
display(plot)

group = primary_caption[["id", "model"]].groupby("model").count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="All By Model", legend=True)
display(plot)

In [ ]:
# @title Plot Summary: By Model Type

group = accepted_primary[["id", "model", "subreddit"]].groupby(["model"]).count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="Models with most images", legend=True)
display(plot)

group = accepted_thumbnail[["id", "model", "subreddit"]].groupby(["model"]).count().sort_values(by="id",
																								ascending=False)
plot_1 = group.plot.bar(figsize=(20, 10), title="Models with most images", legend=True)
display(plot)

In [ ]:
# #@title Disconnect
#
# from google.colab import runtime
# runtime.unassign()